## WIKI WITH RAG TOOL EXECUTION 

updation: use 2 or more rag chain 

In [1]:
# wiki tool 
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [18]:
# tavily  search TOOL 
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API"]=os.getenv("TAVILY_API")

In [32]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_api_key = os.getenv("TAVILY_API")
if not tavily_api_key:
    raise ValueError("TAVILY_API_KEY environment variable is not set.")

tavily_tool = TavilySearchResults(k=5, tavily_api_key=tavily_api_key)


In [30]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [33]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
os.environ['HF_TOCKEN']=os.getenv("HF_TOCKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")


In [34]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [35]:
# retriever tool 
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, embeddings)
retriever=vectordb.as_retriever()
retriever

C:\Users\Gaming\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EEA3F73380>, search_kwargs={})

In [36]:
# create own tools bad on own created retriever
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ") # retriever , retrievername. description

retriever_tool.name

'langsmith-search'

In [37]:
tools=[wiki,retriever_tool,tavily_tool]

In [38]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

C:\Users\Gaming\AppData\Roaming\Python\Python312\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [39]:
## Agents promt and llm chains
# you cannot chain tools in prompt templete so need to call this tool agent to chain tools with llm and prompt 
# make tools agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [40]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [41]:
agent_executor.invoke({"input":"Tell me about Langsmith and how to use it with LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'What is Langsmith and how to use it with LangChain?'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all sta

C:\Users\Gaming\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Invoking: `langsmith-search` with `{'query': 'How can I use Langsmith with LangChain?'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, t

C:\Users\Gaming\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


It seems that Langsmith is a platform for building production-grade LLM applications, and it can be used with LangChain. Langsmith provides features such as observability, evaluation, and prompt engineering to help you build and optimize your LLM applications.

> Finished chain.


{'input': 'Tell me about Langsmith and how to use it with LangChain?',
 'output': 'It seems that Langsmith is a platform for building production-grade LLM applications, and it can be used with LangChain. Langsmith provides features such as observability, evaluation, and prompt engineering to help you build and optimize your LLM applications.'}

In [42]:
agent_executor.invoke({"input":"Tell me about Indian cricket?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Indian cricket team'}`


Page: India national cricket team
Summary: The India men's national cricket team, also known as Men in Blue, represents India in international cricket. It is governed by the Board of Control for Cricket in India (BCCI) and is a full member nation of The India national cricket team, also known as the Men in Blue, represents India in international cricket. It is governed by the Board of Control for Cricket in India (BCCI) and is a full member nation of the International Cricket Council (ICC). The team has a rich history of producing some of the best cricketers in the world, including Sachin Tendulkar, Sunil Gavaskar, Kapil Dev, and many others.

> Finished chain.


{'input': 'Tell me about Indian cricket?',
 'output': 'The India national cricket team, also known as the Men in Blue, represents India in international cricket. It is governed by the Board of Control for Cricket in India (BCCI) and is a full member nation of the International Cricket Council (ICC). The team has a rich history of producing some of the best cricketers in the world, including Sachin Tendulkar, Sunil Gavaskar, Kapil Dev, and many others.'}

In [43]:
agent_executor.invoke({"input":"Tell me about todays ai news"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'AI news todays'}`


Page: Perplexity AI
Summary: Perplexity AI, Inc., or simply Perplexity, is an American privately held software company offering a web search engine that processes user queries and synthesizes responses. It uses large language models and incorporates It seems like the tool call yielded some information about Perplexity AI, but it doesn't seem directly related to "AI news today". Let me try to use another tool to get more relevant information.

> Finished chain.


{'input': 'Tell me about todays ai news',
 'output': 'It seems like the tool call yielded some information about Perplexity AI, but it doesn\'t seem directly related to "AI news today". Let me try to use another tool to get more relevant information.'}

In [44]:
agent_executor.invoke({"input":"What happened at the last wimbledon"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': '2022 Wimbledon Championships'}`


Page: 2022 Wimbledon Championships
Summary: The 2022 Wimbledon Championships was a Grand Slam tier tennis tournament that took place at the All England Lawn Tennis and Croquet Club in Wimbledon, London, United Kingdom. Novak Djokovic successfully defNovak Djokovic successfully defended his title, winning his 21st Grand Slam singles title and his 7th Wimbledon title, by defeating Nick Kyrgios in the final with a score of 4-6, 6-3, 6-4, 7-6(7-3).

> Finished chain.


{'input': 'What happened at the last wimbledon',
 'output': 'Novak Djokovic successfully defended his title, winning his 21st Grand Slam singles title and his 7th Wimbledon title, by defeating Nick Kyrgios in the final with a score of 4-6, 6-3, 6-4, 7-6(7-3).'}

In [46]:
agent_executor.invoke({"input":"after complete data analytics what is the roadmap of data science search in tavilly"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'roadmap of data science search in tavilly'}`


[{'title': 'Tavily - The Web Access Layer for AI Agents', 'url': 'https://tavily.com/', 'content': "If you have any questions, please do not hesitate to reach to our team.\n\nContact Us\n\nWhat is Tavily Search API?\n\nTavily Search API is a specialized search engine designed for Large Language Models (LLMs) and AI agents. It provides real-time, accurate, and unbiased information, enabling AI applications to retrieve and process data efficiently. Tavily is built with AI developers in mind, simplifying the process of integrating dynamic web information into AI-driven solutions. [...] Search is essential for AI assistants. At CopilotKit, we build deeply-integrated AI assistants where search is a critical feature. Tavily's Search API fills this gap, providing accurate, up-to-date information that transforms AI copilots into invaluable productivity

{'input': 'after complete data analytics what is the roadmap of data science search in tavilly',
 'output': 'Based on the results, it seems that Tavily Search API is a key player in the search for data science-related information. The results suggest that Tavily is a search engine designed for Large Language Models (LLMs) and AI agents, providing real-time, accurate, and unbiased information.\n\nOne of the most relevant results is the Medium article "How to Give Your AI Agents Real-Time Search Superpowers with Tavily" which highlights the capabilities of Tavily Search API in providing real-time search results for AI agents.\n\nAnother relevant result is the LangChain documentation for Tavily Search, which shows how to integrate Tavily Search API with LangChain to enable real-time search capabilities for AI agents.\n\nThe results also suggest that there are various tools and frameworks available for data science, such as Python, SQL, and Deep Learning frameworks like TensorFlow, PyTorch